In [ ]:
!pip install anthropic

In [2]:
import requests
import os
from datetime import datetime, timedelta, date
from google.colab import userdata
import anthropic
import re
from typing import List, Tuple
from IPython.display import Markdown, display


def pick_daily_news_from_database(url, headers):


# データベースからデータを取得するためのフィルター
    data = {
        "filter": {
            "and": [
                {
                    "property": "flag",
                    "status": {
                        "equals": "pick"
                    }
                },
                {
                    "or": [
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "day"
                            }
                        },
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "week"
                            }
                        },
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "month"
                            }
                        }
                    ]
                }
            ]
        }
    }

    # APIリクエストを送信
    response = requests.post(url, headers=headers, json=data)
    results = response.json()
    return results

def pick_weekly_news_from_database(url, headers):


# データベースからデータを取得するためのフィルター
    data = {
        "filter": {
            "and": [
                {
                    "property": "flag",
                    "status": {
                        "equals": "pick"
                    }
                },
                {
                    "or": [
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "week"
                            }
                        },
                        {
                            "property": "pickup_type",
                            "select": {
                                "equals": "month"
                            }
                        }
                    ]
                }
            ]
        }
    }

    # APIリクエストを送信
    response = requests.post(url, headers=headers, json=data)
    results = response.json()
    return results

def pick_monthly_news_from_database(url, headers):


    # データベースからデータを取得するためのフィルター
    data = {
        "filter": {
            "and": [
                {
                    "property": "flag",
                    "status": {
                        "equals": "pick"
                    }
                },
                {
                    "property": "pickup_type",
                    "select": {
                        "equals": "month"
                    }
                }
            ]
        }

    }


    # APIリクエストを送信
    response = requests.post(url, headers=headers, json=data)
    results = response.json()
    return results

def add_page_to_database(page_title, database_key, news_contents):

    NOTION_API_KEY = userdata.get('NOTION_API_KEY')
    DATABASE_ID = userdata.get(database_key)

    url = 'https://api.notion.com/v1/pages'

    headers =  {
        'Notion-Version': '2022-06-28',
        'Authorization': 'Bearer ' + NOTION_API_KEY,
        'Content-Type': 'application/json',
    }

    json_data = {
        'parent': { 'database_id': DATABASE_ID },
        'properties': {
            'name': {
                'title': [
                    {
                        'text': {
                            'content': page_title
                        }
                    }
                ]
            },
        },
    }

    page_response = requests.post('https://api.notion.com/v1/pages', headers=headers, json=json_data)
    new_page_id = page_response.json()['id']  # 新しく作成されたページのIDを取得
    return new_page_id

def add_content_to_page(headers, new_page_id, text):


    # ページにブロック（テキスト）を追加
    block_data = {
        "children": [
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": text
                            }
                        }
                    ]
                }
            }
        ]
    }

    block_response = requests.patch(f'https://api.notion.com/v1/blocks/{new_page_id}/children', headers=headers, json=block_data)

def split_text(long_text, length=2000):
    return [long_text[i:i+length] for i in range(0, len(long_text), length)]

def main(pickup_type, title_date, title_name):
    NOTION_API_KEY = userdata.get('NOTION_API_KEY')
    DATABASE_ID = userdata.get('PICKUP_DATABASE_KEY')

    url = f'https://api.notion.com/v1/databases/{DATABASE_ID}/query'

    headers =  {
        'Notion-Version': '2022-06-28',
        'Authorization': 'Bearer ' + NOTION_API_KEY,
        'Content-Type': 'application/json',
    }

    today = date.today()
    formatted_date = today.strftime("%Y-%m-%d")

    if pickup_type == "day":
        update_database_key = "DAILY_DATABASE_KEY"
        results = pick_daily_news_from_database(url, headers)
        page_title_header = "daily_news"
    elif pickup_type == "week":
        update_database_key = "WEEKLY_DATABASE_KEY"
        results = pick_weekly_news_from_database(url, headers)
        page_title_header = "weekly_news"
    elif pickup_type == "month":
        update_database_key = "MONTHLY_DATABASE_KEY"
        results = pick_monthly_news_from_database(url, headers)
        page_title_header = "monthly_news"

    contents_list = []
    for result in results["results"]:
        properties = result["properties"]
        news_url = properties["URL"]["url"]
        tag = properties["tag"]["select"]["name"]
        if not properties["summary"]["rich_text"]:
            print("summaryが空です")
            abstract = ""
        else:
            abstract = properties["summary"]["rich_text"][0]["text"]["content"]
        title = properties["name"]["title"][0]["text"]["content"]
        content = [tag, title, news_url, abstract]
        contents_list.append(content)

    sorted_contents_list = sorted(contents_list, key=lambda x: x[0], reverse=True)

    news_contents = ""
    for content in sorted_contents_list:
        tag = content[0]
        title = content[1]
        news_url = content[2]
        abstract = content[3]

        news_contents += f"\n【{tag}】 {title} {news_url} \n {abstract} \n --------------------------------"


    page_title = f"{page_title_header}{title_date}{title_name}"


    new_page_id = add_page_to_database(page_title, update_database_key, news_contents)

    contents = split_text(news_contents)

    for content in contents:
        add_content_to_page(headers, new_page_id, content)

    return news_contents

def split_news_articles(text: str) -> List[str]:
  """
  入力テキストを個別のニュース記事に分割する

  Args:
      text (str): 入力テキスト

  Returns:
      List[str]: 個別のニュース記事のリスト
  """
  articles = text.split('-----------------------------')
  return [article.strip() for article in articles if article.strip()]

def process_individual_news(client: anthropic.Client, article: str) -> str:
    """
    個別のニュース記事を処理する

    Args:
        client: Anthropic APIクライアント
        article (str): 1つのニュース記事

    Returns:
        str: 処理済みの記事（タイトルと要約）
    """
    system_prompt = """
    以下のニュース記事を処理してください：
    1. 20語程度の一文でタイトルを作成
    2. 200文字程度、2-3文で要約
    3. 以下の形式で出力：

    [OUTPUT_START]
    作成したニュースタイトル
    【タグ】ニュースの要約文 ニュースのURL
    [OUTPUT_END]

    ・元テキストの【】タグをそのまま使用
    ・ですます調で記述
    """

    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0,
        system=system_prompt,
        messages=[{"role": "user", "content": article}]
    )

    response = message.content[0].text

    # 出力を抽出
    start_idx = response.find('[OUTPUT_START]') + len('[OUTPUT_START]')
    end_idx = response.find('[OUTPUT_END]')
    return response[start_idx:end_idx].strip()

def generate_title_and_impression(client: anthropic.Client, summaries: str) -> Tuple[str, str]:
    """
    全ての要約からタイトルと感想を生成

    Args:
        client: Anthropic APIクライアント
        summaries (str): 全ての記事の要約

    Returns:
        Tuple[str, str]: (総合タイトル, 感想)
    """
    system_prompt = """
    以下のニュース要約から、総合タイトルと感想を生成してください。

    タイトル:
    ・形式：「[主要ニュースのポイントをカンマ区切りで6つ程度]｜[月日]のIT・AIニュースピックアップ！」
    ・重要ニュース、インパクトの大きな出来事、新しい潮流を優先
    ・OpenAI, Google, Anthropic, Microsoft, Amazon, ByteDance,Alibabaのニュースは必ず入れてください

    感想:
    ・タイトルで選んだキーワードを盛り込む
    ・親しみやすく優しいが丁寧な人柄が感じられる文体
    ・600文字程度

    必ず以下の形式で出力してください：
    [TITLE_START]
    （総合タイトル）
    [TITLE_END]

    [IMPRESSION_START]
    （ニュースの感想）
    [IMPRESSION_END]
    """

    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=2000,
        temperature=0,
        system=system_prompt,
        messages=[{"role": "user", "content": summaries}]
    )

    response = message.content[0].text

    # タイトルと感想を抽出
    title = response[response.find('[TITLE_START]')+len('[TITLE_START]'):response.find('[TITLE_END]')].strip()
    impression = response[response.find('[IMPRESSION_START]')+len('[IMPRESSION_START]'):response.find('[IMPRESSION_END]')].strip()

    return title, impression

def process_all_news(input_text: str) -> Tuple[str, str, str]:
    """
    全てのニュースを処理する

    Args:
        input_text (str): 入力テキスト

    Returns:
        Tuple[str, str, str]: (ニュースまとめ, 総合タイトル, 感想)
    """
    api_key = userdata.get('CLAUDE_API_KEY')
    client = anthropic.Client(api_key=api_key)

    # ニュースを分割
    articles = split_news_articles(input_text)

    # 各ニュースを個別に処理
    processed_articles = []
    for article in articles:
        try:
            processed = process_individual_news(client, article)
            processed_articles.append(processed)
        except Exception as e:
            print(f"記事の処理中にエラーが発生: {str(e)}")

    # 全ての処理済み記事を結合
    all_summaries = "\n\n".join(processed_articles)

    # タイトルと感想を生成
    title, impression = generate_title_and_impression(client, all_summaries)

    return all_summaries, title, impression

def format_news_articles_for_note(text):
    """
    ニュース記事のテキストをnoteのエディタ向けに整形する関数
    様々なタグに対応し、完全に独立したブロックを作成します

    Args:
        text (str): 複数のニュース記事を含む入力テキスト

    Returns:
        str: note用に整形されたニュース記事テキスト
    """
    # システムの改行コードを使用
    newline = os.linesep

    # URLで記事を分割するパターン
    # URLの後に改行または文字列終端があるものを検出
    pattern = r'(.*?https://[^\s]+)(?:\n|$)'

    # 記事を抽出
    articles = re.findall(pattern, text, re.DOTALL)

    formatted_articles = []
    for article in articles:
        # URLを抽出
        url_pattern = r'(https://[^\s]+)$'
        url_match = re.search(url_pattern, article)
        if not url_match:
            continue

        url = url_match.group(1)

        # URLを除いた部分を取得
        content = article[:url_match.start()].strip()

        # タグパターンを検出（【...】の形式）
        tag_pattern = r'【[^】]+】'

        # 最初のタグの位置を検索
        tag_matches = list(re.finditer(tag_pattern, content))
        if not tag_matches:
            continue

        # タイトルは最初のタグの前まで
        first_tag_pos = tag_matches[0].start()
        title = content[:first_tag_pos].strip()

        # タグと説明文を抽出
        tag_content = content[first_tag_pos:].strip()

        # 段落区切りを使用して整形
        formatted_parts = [
            f"### {title}",
            tag_content,
            url
        ]

        # 各パートを段落区切りで結合
        formatted_article = f"{newline}{newline}".join(formatted_parts)
        formatted_articles.append(formatted_article)

    # 記事間により大きな区切りを入れる
    article_separator = f"{newline}{newline}{newline}"
    return article_separator.join(formatted_articles)

def display_news_markdown(news_title, news_impression, formatted_result):
    """
    ニュース関連の出力をマークダウン形式で表示する

    Args:
        news_title (str): ニュースのタイトル
        news_impression (str): ニュースの感想
        formatted_result (str): フォーマット済みのニュース本文
    """
    markdown_content = f"""# {news_title}

こんにちは、MOFUです。

{news_impression}

{formatted_result}
"""
    display(Markdown(markdown_content))


In [ ]:
pickup_type = "day"
title_date = "2025-01-25"
title_name = ""

# Notionデータベースからflag='true'でpickup_typeに合致するものを抽出
news_contents = main(pickup_type, title_date, title_name)

# Claude APIを使って、ニュースの要約・整形、ブログタイトル、ブログの感想を作成
news_summary, news_title, news_impression = process_all_news(news_contents)

# noteに貼り付ける用の整形
formatted_result = format_news_articles_for_note(news_summary)

# マークダウン形式で結果を出力（出力結果をnoteにコピペすればOK）
display_news_markdown(
    news_title=news_title,
    news_impression=news_impression,
    formatted_result=formatted_result
)
